<a href="https://colab.research.google.com/github/slbryson/MLlearn/blob/master/Para_WhisperGPUTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Grab the whisper library
!pip install -U openai-whisper
!pip install git+https://github.com/openai/whisper.git 

In [ ]:
import torch
import math
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
import scipy.io.wavfile as wavfile
import multiprocessing
import random


In [ ]:

import whisper

# Load the large model
model_type = 'medium'
model1 = whisper.load_model(model_type)
model2 = whisper.load_model(model_type)


In [ ]:
from gtts import gTTS 

# The text that you want to convert to audio
mytext = 'Hello, welcome to my channel. I hope you are doing well. '
language = 'en'
speech = gTTS(text = mytext, lang = language, slow = False)

# Saving the converted audio in a mp3 file named
speech.save("random_audio.mp3")
# Saving the converted audio in a wav file named random_audio.wav
speech.save("random_audio.wav")


In [ ]:

# Check if CUDA is available
if torch.cuda.is_available():
    # Use the first available GPU device
    device = torch.device("cuda:0")
    # Print the current device
    print(f"Using device: {torch.cuda.get_device_name(device)}")
elif torch.backends.mps.is_built():
    # this ensures that the current MacOS version is at least 12.3+
    print(torch.backends.mps.is_available())
    # this ensures that the current current PyTorch installation was built with MPS activated.
    print(torch.backends.mps.is_built())
    device = torch.device("mps")
    print("Using MPS device")

else:
    print("CUDA is not available, using CPU instead.")
    device = torch.device("cpu")


In [ ]:


def benchmark_model(model, input_tensor, device, num_iterations):
    # Move the model to the current device
    model.to(device)

    # Move the input tensor to the current device
    input_tensor = input_tensor.to(device)

    # Run the model once to initialize the GPU
    _ = model.transcribe(input_tensor)

    # Benchmark the GPU by running the model multiple times
    times = []
    for i in range(num_iterations):
        start_time = time.time()
        _ = model.transcribe(input_tensor)
        end_time = time.time()
        times.append(end_time - start_time)

    # Calculate the average time per iteration
    avg_time_per_iteration = sum(times) / num_iterations

    # Print the average time per iteration
    print(f"Average time per iteration: {avg_time_per_iteration:.5f} seconds")

    # Return the average time per iteration
    return avg_time_per_iteration


Load the model

In [ ]:

# Set the Length of the audio
audio_len = 5

# Load the audio file using whisper
input_tensor = whisper.load_audio('random_audio.mp3')
input_tensor = whisper.pad_or_trim(input_tensor)
# Reshape the input tensor to have shape (1, num_samples, num_channels)
# input_tensor = input_tensor.reshape((1, -1, 1))

# Move the input tensor to the current device
input_tensor = torch.from_numpy(input_tensor).to(device)




In [ ]:
# Check the Input
print(input_tensor.shape)
# Get the output from model1 and model2 and print the results from both the models 
# Set the environment variable PYTORCH_ENABLE_MPS_FALLBACK=1 to enable MPS fallback
# Set the environment variable PYTORCH_ENABLE_MPS=1 to enable MPS
# Set the environment variable PYTORCH_ENABLE_MPS=0 to disable MPS
# Set the environment variable PYTORCH_ENABLE_MPS_FALLBACK=0 to disable MPS fallback

#python set an environment variable in MacOS 
import os
os.environ["PYTORCH_ENABLE_MPS"] = "0"
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
# print the environment variable PYTORCH_ENABLE_MPS
print(os.environ["PYTORCH_ENABLE_MPS"])
#print the environment variable PYTORCH_ENABLE_MPS_FALLBACK 
print(os.environ["PYTORCH_ENABLE_MPS_FALLBACK"])


In [ ]:
result = benchmark_model(model2,input_tensor,device,1)


In [ ]:
avg_time = benchmark_model(model1,input_tensor,device,1)

In [ ]:
inputs= [model1,input_tensor,device,2]
# avg1 =[]
with multiprocessing.Pool(processes=2) as pool:
  avg1 = pool.imap(benchmark_model,inputs)

# print(avg1)

In [ ]:
print(avg1)

In [ ]:

# Number of Iterations
num_iter = 1
# Create two processes to run the models in parallel
process1 = multiprocessing.get_context("spawn").Process(target=benchmark_model, args=(model1, input_tensor, device, num_iter))
process2 = multiprocessing.get_context("spawn").Process(target=benchmark_model, args=(model2, input_tensor, device, num_iter))

# Start the processes
process1.start()
process2.start()

# Wait for the processes to finish
process1.join()
process2.join()
